In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.callbacks import EarlyStopping, ModelCheckpoint


df = pd.read_csv("", parse_dates=['date'], dayfirst=True)
df.set_index('date', inplace=True)

data = df['x'].values.reshape(-1, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)


split_idx = int(len(scaled_data) * 0.8)
train_data, test_data = scaled_data[:split_idx], scaled_data[split_idx:]

time_step = 1


def create_dataset(data, time_step):
    X, y = [], []
    for i in range(len(data) - time_step):
        X.append(data[i:(i + time_step), 0])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

model = Sequential([
    LSTM(units=100, return_sequences=True, input_shape=(time_step, 1)),
    LSTM(units=100),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_lstm_model.keras', monitor='val_loss', save_best_only=True, mode='min')

model.fit(X_train, y_train, epochs=20, batch_size=16, validation_split=0.1, 
          callbacks=[early_stopping, model_checkpoint], verbose=1)

y_pred = model.predict(X_test)

y_pred = scaler.inverse_transform(y_pred)
y_test = scaler.inverse_transform(y_test.reshape(-1, 1))

plt.figure(figsize=(14, 7))

plt.plot(df.index[:split_idx], data[:split_idx], label='Eğitim Verisi', color='blue', linewidth=1)

plt.plot(df.index[split_idx:], data[split_idx:], label='Test Verisi', color='green', linewidth=1)

test_dates = df.index[split_idx + time_step:]
plt.plot(test_dates, y_pred, label='Test Tahminleri', color='purple', linestyle='-')

plt.xlabel('Tarih')
plt.ylabel('x')
plt.title('LSTM Tahminleri')
plt.legend()
plt.grid(True)
plt.show()
